In [38]:
import requests
import json
import pandas as pd
import datetime
import numpy as np
import time
from bs4 import BeautifulSoup

In [137]:
with open('/Users/ChristopherKuzemka/Documents/GA/dsi_11/projects/capstone/env.json') as f:
    information = json.load(f)

In [138]:
host_name = information.get('x-rapidapi-host')
apiKey = information.get('x-rapidapi-key')

Please note I am limited to 50 requests per minute.

1) Browse Quotes - Returns the cheapest quotes that meet your query. The prices come from our cached prices resulting from our users’ searches.

2) Browse Routes - Similar to Browse Quotes but with the quotes grouped by routes. This provides the cheapest destinations (countries, cities or airports) from our cached data.

3) Browse Dates - Similar to Browse Quotes but with the quotes grouped by outbound and inbound date. Useful to find the lowest price for a given route, over either a month or a 12 month period.

4) Browse Grid - Similar to Browse Dates but with some pre-processing on our side to output a two-dimensional array to easily display the response in calendar format.

# Data Prep

In [40]:
current_flights = pd.read_csv('../data/current_flights.csv')
current_flights

,Unnamed: 0,faFlightID,ident,prefix,type,suffix,origin,destination,timeout,timestamp,...,lowLatitude,highLongitude,highLatitude,groundspeed,altitude,heading,altitudeStatus,updateType,altitudeChange,waypoints
0,0,DAL333-1590465975-fa-0008,DAL333,NaN,A321,NaN,KATL,KLAX,0,1590716390,...,32.94676,-84.44664,33.70005,448,300,264,NaN,A,D,33.64 -84.43 33.68 -84.28 33.81 -84.28 33.81 -...
1,1,KLM601-1590468354-airline-0005,KLM601,NaN,B77W,NaN,EHAM,KLAX,0,1590711509,...,33.95142,4.71741,66.06976,130,1,263,NaN,A,D,52.31 4.76 53.02 2.53 53.06 2.46 53.2 1.53 53....
2,2,VIR607-1590664542-ed-0002,VIR607,NaN,B789,NaN,EGLL,KLAX,0,1590711368,...,33.95091,-0.39345,66.09780,101,1,263,NaN,A,D,51.48 -0.46 51.56 -0.59 51.62 -0.68 51.7 -0.79...
3,3,DAL702-1590465982-fa-0006,DAL702,NaN,A321,NaN,KATL,KLAX,0,1590709847,...,33.64682,-84.44602,35.52932,117,1,263,NaN,A,D,33.64 -84.43 33.68 -84.28 33.79 -85 33.81 -85....
4,4,ACA572-1590468353-airline-0278,ACA572,NaN,A319,NaN,CYVR,KLAX,0,1590706589,...,33.95183,-118.17553,49.18984,109,1,263,NaN,A,D,49.19 -123.18 49.12 -123.26 49.12 -123.27 49.0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,100,EDV4708-1590468354-airline-0112,EDV4708,NaN,CRJ9,NaN,KAGS,KATL,0,1590664135,...,33.19607,-81.97964,33.63222,126,10,91,NaN,A,D,33.37 -81.96 33.39 -82.03 33.39 -82.04 33.38 -...
101,101,ABW341-1590583043-eb-0002,ABW341,NaN,B744,NaN,EBLG,KATL,0,1590631398,...,33.53333,5.56940,54.10380,132,9,90,,A,D,50.64 5.44 50.69 5.24 50.73 5.08 50.78 4.89 50...
102,102,DLH8176-1590641113-airline-0180,DLH8176,NaN,B772,NaN,EDDF,KATL,0,0,...,200.00000,-200.00000,-200.00000,0,0,0,NaN,NaN,NaN,NaN
103,103,CLX8626-1590641113-airline-0361,CLX8626,NaN,B744,NaN,ELLX,KATL,0,0,...,200.00000,-200.00000,-200.00000,0,0,0,NaN,NaN,NaN,NaN


In [81]:
flight_schedules = pd.read_csv('../data/flight_schedules.csv')
flight_schedules

,Unnamed: 0,ident,actual_ident,departuretime,arrival_time,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach
0,0,UAL4282,ASQ4282,1588330800,1588340820,CYUL,KORD,E75L,Business: Refreshments / Economy: Food for sale,0,12,58
1,1,ACA7591,AC27591,1588335000,1588343880,CYUL,KORD,E75L,"Business: Breakfast / Economy: Breakfast, Food...",0,12,64
2,2,UAL8371,AC27591,1588335000,1588343880,CYUL,KORD,E75L,Business: Breakfast / Economy: Food for sale,0,12,64
3,3,UAL4245,ASQ4245,1588341060,1588351080,CYUL,KORD,E75L,Business: Refreshments / Economy: Food for sale,0,12,58
4,4,UAL8481,AC27595,1588353300,1588362000,CYUL,KORD,E75L,Business: Meal / Economy: Food for sale,0,12,64
...,...,...,...,...,...,...,...,...,...,...,...,...
5818,5818,UAL464,NaN,1590514200,1590523860,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114
5819,5819,SWA378,NaN,1590520500,1590529500,KDEN,KPDX,B738,Economy: No meal,0,0,175
5820,5820,DLH9070,UAL393,1590539460,1590549120,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114
5821,5821,UAL393,NaN,1590539460,1590549120,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114


In [82]:
flight_combinations = pd.read_csv('../data/flight_combinations.csv')
flight_combinations

,Unnamed: 0,origin,destination,0
0,0,CYHM,KJFK,1
1,1,CYUL,KORD,1
2,2,CYVR,KLAX,1
3,3,CYYZ,KIAH,2
4,4,CYYZ,KJFK,1
5,5,CYYZ,KLAX,1
6,6,CYYZ,KORD,4
7,7,EBBR,KMIA,2
8,8,EBBR,KORD,1
9,9,EBLG,KATL,1


In [ ]:
flight_combinations['origin'].unique()

In [49]:
len(flight_combinations['origin'].unique())

33

In [48]:
flight_combinations['destination'].unique()

array(['KJFK', 'KORD', 'KLAX', 'KIAH', 'KMIA', 'KATL', 'KPDX'],
      dtype=object)

Will have to programmatically find a way to create a bunch if IATA codes out of the ICAO codes shown here. There are only 33 unique origins and 7 unique destinations making a total of 40 different airports studied. In total, we have 59 combinations studied. 

## Making the IATA Dictionaries

In [51]:
origin_IATA_dict = {k:[] for k in flight_combinations['origin'].unique()}

In [65]:
destination_IATA_dict = {k:[] for k in flight_combinations['destination'].unique()}

In [56]:
origin_IATA_list = ['YHM', 'YUL', 'YVR', 'YYZ', 'BRU', 'LGG', 'FRA', 'NUE', 'LEJ', 'LHR', 'AMS', 'MST', 'DUB', 'LUX', 'ABQ', 'ABY', 'AEX', 'AFW', 'AGS', 'APF', 'AST', 'ATL', 'AUS', 'BFI', 'BIH', 'BKV', 'BOI', 'BOS', 'BUR', 'BWI', 'CVG', 'CVO', 'DEN']

In [67]:
destination_IATA_list = ['JFK', 'ORD', 'LAX', 'IAH', 'MIA', 'ATL', 'PDX']

In [63]:
for k,v in enumerate(origin_IATA_dict):
    origin_IATA_dict[v].append(origin_IATA_list[k])

In [69]:
for k,v in enumerate(destination_IATA_dict):
    destination_IATA_dict[v].append(destination_IATA_list[k])

### 

## Converts Time to Datetime

In [257]:
flight_schedules['departure_date'] = [datetime.datetime.fromtimestamp(flight_schedules.loc[k, 'departuretime']).strftime('%Y-%m') for k in range(len(flight_schedules))]

In [258]:
flight_schedules['future_departures'] = flight_schedules['departure_date'].str.replace('2020', '2021')

In [260]:
flight_schedules

,Unnamed: 0,ident,actual_ident,departuretime,arrival_time,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach,departure_date,future_departures,destination_IATA,origin_IATA
0,0,UAL4282,ASQ4282,1588330800,1588340820,CYUL,KORD,E75L,Business: Refreshments / Economy: Food for sale,0,12,58,2020-05,2021-05,ORD,YUL
1,1,ACA7591,AC27591,1588335000,1588343880,CYUL,KORD,E75L,"Business: Breakfast / Economy: Breakfast, Food...",0,12,64,2020-05,2021-05,ORD,YUL
2,2,UAL8371,AC27591,1588335000,1588343880,CYUL,KORD,E75L,Business: Breakfast / Economy: Food for sale,0,12,64,2020-05,2021-05,ORD,YUL
3,3,UAL4245,ASQ4245,1588341060,1588351080,CYUL,KORD,E75L,Business: Refreshments / Economy: Food for sale,0,12,58,2020-05,2021-05,ORD,YUL
4,4,UAL8481,AC27595,1588353300,1588362000,CYUL,KORD,E75L,Business: Meal / Economy: Food for sale,0,12,64,2020-05,2021-05,ORD,YUL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5818,5818,UAL464,NaN,1590514200,1590523860,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114,2020-05,2021-05,PDX,DEN
5819,5819,SWA378,NaN,1590520500,1590529500,KDEN,KPDX,B738,Economy: No meal,0,0,175,2020-05,2021-05,PDX,DEN
5820,5820,DLH9070,UAL393,1590539460,1590549120,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114,2020-05,2021-05,PDX,DEN
5821,5821,UAL393,NaN,1590539460,1590549120,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114,2020-05,2021-05,PDX,DEN


## Adds IATA codes

In [261]:
flight_schedules['destination_IATA'] = flight_schedules['destination'].map(destination_IATA_dict)

In [262]:
flight_schedules['origin_IATA'] = flight_schedules['origin'].map(origin_IATA_dict)

In [264]:
flight_schedules

,Unnamed: 0,ident,actual_ident,departuretime,arrival_time,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach,departure_date,future_departures,destination_IATA,origin_IATA
0,0,UAL4282,ASQ4282,1588330800,1588340820,CYUL,KORD,E75L,Business: Refreshments / Economy: Food for sale,0,12,58,2020-05,2021-05,[ORD],[YUL]
1,1,ACA7591,AC27591,1588335000,1588343880,CYUL,KORD,E75L,"Business: Breakfast / Economy: Breakfast, Food...",0,12,64,2020-05,2021-05,[ORD],[YUL]
2,2,UAL8371,AC27591,1588335000,1588343880,CYUL,KORD,E75L,Business: Breakfast / Economy: Food for sale,0,12,64,2020-05,2021-05,[ORD],[YUL]
3,3,UAL4245,ASQ4245,1588341060,1588351080,CYUL,KORD,E75L,Business: Refreshments / Economy: Food for sale,0,12,58,2020-05,2021-05,[ORD],[YUL]
4,4,UAL8481,AC27595,1588353300,1588362000,CYUL,KORD,E75L,Business: Meal / Economy: Food for sale,0,12,64,2020-05,2021-05,[ORD],[YUL]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5818,5818,UAL464,NaN,1590514200,1590523860,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114,2020-05,2021-05,[PDX],[DEN]
5819,5819,SWA378,NaN,1590520500,1590529500,KDEN,KPDX,B738,Economy: No meal,0,0,175,2020-05,2021-05,[PDX],[DEN]
5820,5820,DLH9070,UAL393,1590539460,1590549120,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114,2020-05,2021-05,[PDX],[DEN]
5821,5821,UAL393,NaN,1590539460,1590549120,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114,2020-05,2021-05,[PDX],[DEN]


In [265]:
flight_schedules['destination_IATA'] = [ls[0] for ls in flight_schedules['destination_IATA']]

In [266]:
flight_schedules['origin_IATA'] = [ls[0] for ls in flight_schedules['origin_IATA']]

In [267]:
type(flight_schedules.loc[0, 'future_departures'])

str

# API playground

## Testing Quotes

__Quotes Inputs:__

- Country (Required)

- Currency (Required)

- Locale (Required)

- originplace (Required)

- destinationplace (Required)

- outboundpartialdate (Required)

- inboundpartialdate (Optional)

In [268]:
def get_prices(country, currency, locale, flight_data):
    start_time = time.time() #epoch start time
    output_list = []
    count = 0


    for k in range(len(flight_data['origin'])):
        url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/{country}/{currency}/{locale}/{flight_data.loc[k,'origin_IATA']}-sky/{flight_data.loc[k, 'destination_IATA']}-sky/{flight_data.loc[k, 'future_departures']}"

        querystring = {"inboundpartialdate":"anytime"}

        headers = {
            'x-rapidapi-host': host_name,
            'x-rapidapi-key': apiKey
        }

        response = requests.get(url = url, headers=headers, params=querystring)
        print(f"Getting the data for: (Origin: {flight_data.loc[k,'origin_IATA']}), (Destination: {flight_data.loc[k, 'destination_IATA']}), (For Date: {flight_data.loc[k, 'future_departures']}) ")
        json_response = response.json()
        output_list.append(json_response)
        count = count + 1
        if count <= 48:
            continue
        else:
            print("Approaching request limit. Sleeping for 60 seconds....")
            print(f"The request count index you are at is: {k}")
            print(f"The size of the data file you are requesting off of is: {len(flight_data)} ")
            print(f"This search is {(k/len(flight_data)) * 100} percent complete.")
            print(f"Elapsed time is: {time.time() - start_time} seconds.")
            time.sleep(60)
            print("Conducting another request batch.")
            
            count = 0
    print(f"Elapsed time of process is: {time.time() - start_time}")
    print(f"The start time was: {start_time}")
    print(f"The end time is: {time.time()}")
    return pd.DataFrame(output_list)
        


In [253]:
short_schedules = flight_schedules[0:50]

In [269]:
flight_schedules

,Unnamed: 0,ident,actual_ident,departuretime,arrival_time,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach,departure_date,future_departures,destination_IATA,origin_IATA
0,0,UAL4282,ASQ4282,1588330800,1588340820,CYUL,KORD,E75L,Business: Refreshments / Economy: Food for sale,0,12,58,2020-05,2021-05,ORD,YUL
1,1,ACA7591,AC27591,1588335000,1588343880,CYUL,KORD,E75L,"Business: Breakfast / Economy: Breakfast, Food...",0,12,64,2020-05,2021-05,ORD,YUL
2,2,UAL8371,AC27591,1588335000,1588343880,CYUL,KORD,E75L,Business: Breakfast / Economy: Food for sale,0,12,64,2020-05,2021-05,ORD,YUL
3,3,UAL4245,ASQ4245,1588341060,1588351080,CYUL,KORD,E75L,Business: Refreshments / Economy: Food for sale,0,12,58,2020-05,2021-05,ORD,YUL
4,4,UAL8481,AC27595,1588353300,1588362000,CYUL,KORD,E75L,Business: Meal / Economy: Food for sale,0,12,64,2020-05,2021-05,ORD,YUL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5818,5818,UAL464,NaN,1590514200,1590523860,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114,2020-05,2021-05,PDX,DEN
5819,5819,SWA378,NaN,1590520500,1590529500,KDEN,KPDX,B738,Economy: No meal,0,0,175,2020-05,2021-05,PDX,DEN
5820,5820,DLH9070,UAL393,1590539460,1590549120,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114,2020-05,2021-05,PDX,DEN
5821,5821,UAL393,NaN,1590539460,1590549120,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114,2020-05,2021-05,PDX,DEN


In [270]:
len(flight_schedules)

5823

In [271]:
test_list = get_prices('US', 'USD', 'en-US', flight_schedules)

equest count index you are at is: 5585
The size of the data file you are requesting off of is: 5823 
This search is 95.91275974583549 percent complete.
Elapsed time is: 10226.420287132263 seconds.
Conducting another request batch.
Getting the data for: (Origin: BOS), (Destination: MIA), (For Date: 2021-05) 
Getting the data for: (Origin: BOS), (Destination: MIA), (For Date: 2021-05) 
Getting the data for: (Origin: BOS), (Destination: MIA), (For Date: 2021-05) 
Getting the data for: (Origin: BOS), (Destination: MIA), (For Date: 2021-05) 
Getting the data for: (Origin: BOS), (Destination: MIA), (For Date: 2021-05) 
Getting the data for: (Origin: BOS), (Destination: MIA), (For Date: 2021-05) 
Getting the data for: (Origin: BOS), (Destination: MIA), (For Date: 2021-05) 
Getting the data for: (Origin: BOS), (Destination: MIA), (For Date: 2021-05) 
Getting the data for: (Origin: BOS), (Destination: MIA), (For Date: 2021-05) 
Getting the data for: (Origin: BOS), (Destination: MIA), (For Date:

In [276]:
test_list.to_csv('../data/may_pricing_per_flight.csv')

In [251]:
test_df

,Quotes,Places,Carriers,Currencies
0,"[{'QuoteId': 1, 'MinPrice': 175.0, 'Direct': T...","[{'PlaceId': 73076, 'IataCode': 'ORD', 'Name':...","[{'CarrierId': 835, 'Name': 'Air Canada'}]","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."
1,"[{'QuoteId': 1, 'MinPrice': 175.0, 'Direct': T...","[{'PlaceId': 73076, 'IataCode': 'ORD', 'Name':...","[{'CarrierId': 835, 'Name': 'Air Canada'}]","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."
2,"[{'QuoteId': 1, 'MinPrice': 175.0, 'Direct': T...","[{'PlaceId': 73076, 'IataCode': 'ORD', 'Name':...","[{'CarrierId': 835, 'Name': 'Air Canada'}]","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."
3,"[{'QuoteId': 1, 'MinPrice': 175.0, 'Direct': T...","[{'PlaceId': 73076, 'IataCode': 'ORD', 'Name':...","[{'CarrierId': 835, 'Name': 'Air Canada'}]","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."
4,"[{'QuoteId': 1, 'MinPrice': 175.0, 'Direct': T...","[{'PlaceId': 73076, 'IataCode': 'ORD', 'Name':...","[{'CarrierId': 835, 'Name': 'Air Canada'}]","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."
5,"[{'QuoteId': 1, 'MinPrice': 175.0, 'Direct': T...","[{'PlaceId': 73076, 'IataCode': 'ORD', 'Name':...","[{'CarrierId': 835, 'Name': 'Air Canada'}]","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."
6,"[{'QuoteId': 1, 'MinPrice': 175.0, 'Direct': T...","[{'PlaceId': 73076, 'IataCode': 'ORD', 'Name':...","[{'CarrierId': 835, 'Name': 'Air Canada'}]","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."
7,"[{'QuoteId': 1, 'MinPrice': 175.0, 'Direct': T...","[{'PlaceId': 73076, 'IataCode': 'ORD', 'Name':...","[{'CarrierId': 835, 'Name': 'Air Canada'}]","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."
8,"[{'QuoteId': 1, 'MinPrice': 175.0, 'Direct': T...","[{'PlaceId': 73076, 'IataCode': 'ORD', 'Name':...","[{'CarrierId': 835, 'Name': 'Air Canada'}]","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."
9,"[{'QuoteId': 1, 'MinPrice': 175.0, 'Direct': T...","[{'PlaceId': 73076, 'IataCode': 'ORD', 'Name':...","[{'CarrierId': 835, 'Name': 'Air Canada'}]","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."


In [84]:
response_json = response.json()

In [152]:
response_json

dict